## Fine-tuning VGG-16 model pre-trained with ImageNet data

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [3]:
# Detect GPU device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device : {device}")

Using device : cpu


In [4]:
# Image preprocessing

transform_train = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Loading data
DATA_DIR = "" # To be specified
train_dataset = train_dataset = datasets.ImageFolder(root=f"{DATA_DIR}/train", transform=transform_train)
val_dataset = datasets.ImageFolder(root=f"{DATA_DIR}/val", transform=transform_test)
test_dataset = datasets.ImageFolder(root=f"{DATA_DIR}/test", transform=transform_test)

In [ ]:
# DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load pre-trained model
model = models.vgg16(pretrained=True)
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False
# Modify the last layer
n_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(n_features, 2)
model.to(device)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
# Training loop 
n_epochs = 10
for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
    
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {running_loss/len(train_loader):.4f}")
        scheduler.step()

In [ ]:
# Evaluate the set of test
model.eval()
correct = 0
total = 0
y_true = []
y_pred = []

with torch.no_grad():
        for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                y_true.extend(labels.tolist())
                y_pred.extend(predicted.tolist())

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

In [ ]:
# Matrix of confusion
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Abstract', 'Figurative'], yticklabels=['Abstract', 'Figurative'])
plt.xlabel('Predicted')
plt.ylabel('Real')
plt.title('Matrix of confusion')
plt.show()